# Exploring Rana Ayub Data and Models to mirror our desired strucutre 

#### Author: Ridha Alkhabaz 
#### Topic:
here we want to create multiple methods to parse rana ayub data. Then, we will construct a number of ways to go about it. Finally, we will share our results to the reminder of the team. 

#### imports:

In [225]:
import numpy as np
import pandas as pd 
import torch 
import torch_geometric
from torch_geometric.data import HeteroData
import json
from tqdm import tqdm
from transformers import pipeline, PreTrainedModel, PreTrainedTokenizer, PreTrainedTokenizerFast
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from scipy.special import softmax
import warnings
warnings.filterwarnings("ignore")

In [101]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)


#### Reading and parsing the data 

In [12]:
with open('./data/tweets.json') as f:
    alice = [json.loads(line) for line in f]

In [18]:
with open('./data/involved_tweets.json') as f:
    involved_tweets = [json.loads(line) for line in f]

In [30]:
with open('./data/involved_users.json') as f:
    involved_users = [json.loads(line) for line in f]

In [102]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)

In [103]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [263]:
count = 0
for tweet in involved_tweets[0]:
    if tweet['conversation_id'] =='998871190923104256':
        
        count+=1
        
count
        

193

#### Example of the data 

In [ ]:
invlvtwt = pd.DataFrame(involved_tweets[0])
subsetidxs = list(invlvtwt.conversation_id.unique()[:50])
subsetdf = invlvtwt[invlvtwt.conversation_id.isin(subsetidxs)]

In [272]:
def llm_classify(text):
    record_text = preprocess(text)
    encoded_input = tokenizer(record_text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    return scores
subsetdf['score'] = subsetdf['text'].apply(llm_classify)
subsetdf['label'] = subsetdf['score'].apply(np.argmax)
subsetdf.head()

,referenced_tweets,lang,in_reply_to_user_id,author_id,text,possibly_sensitive,edit_history_tweet_ids,reply_settings,created_at,context_annotations,entities,conversation_id,public_metrics,id,attachments,edit_controls,geo,withheld,score,label
0,"[{'type': 'replied_to', 'id': '159708322018442...",qme,268676434,1398533566619811841,@RanaAyyub @sky_phd https://t.co/aw9ThjoBZ9,False,[1597290445016510464],everyone,2022-11-28T18:04:52.000Z,"[{'domain': {'id': '10', 'name': 'Person', 'de...","{'mentions': [{'start': 0, 'end': 10, 'usernam...",1597010232848154624,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1597290445016510464,"{'media_keys': ['3_1597290439295442944', '3_15...","{'edits_remaining': 5, 'is_edit_eligible': Fal...",NaN,NaN,"[0.028033514, 0.87310696, 0.098859556]",1
1,"[{'type': 'replied_to', 'id': '159701023284815...",und,117673128,268676434,@sky_phd 🙏,False,[1597083220184428544],everyone,2022-11-28T04:21:26.000Z,"[{'domain': {'id': '10', 'name': 'Person', 'de...","{'mentions': [{'start': 0, 'end': 8, 'username...",1597010232848154624,"{'retweet_count': 0, 'reply_count': 2, 'like_c...",1597083220184428544,NaN,"{'edits_remaining': 5, 'is_edit_eligible': Fal...",NaN,NaN,"[0.04223864, 0.43336734, 0.524394]",2
2,"[{'type': 'replied_to', 'id': '159708322018442...",en,268676434,48441905,@RanaAyyub @sky_phd Read it. Found nothing of ...,False,[1597092138692800514],everyone,2022-11-28T04:56:53.000Z,"[{'domain': {'id': '10', 'name': 'Person', 'de...","{'mentions': [{'start': 0, 'end': 10, 'usernam...",1597010232848154624,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",1597092138692800514,NaN,"{'edits_remaining': 5, 'is_edit_eligible': Fal...",NaN,NaN,"[0.7693299, 0.22088087, 0.009789176]",0
3,"[{'type': 'quoted', 'id': '1596721934422593537'}]",en,NaN,117673128,🧵 Must read. https://t.co/tfXqa7G3cO,False,[1597010232848154624],everyone,2022-11-27T23:31:25.000Z,NaN,"{'urls': [{'start': 13, 'end': 36, 'url': 'htt...",1597010232848154624,"{'retweet_count': 2, 'reply_count': 1, 'like_c...",1597010232848154624,NaN,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,NaN,"[0.004439414, 0.13561116, 0.8599494]",2
4,"[{'type': 'replied_to', 'id': '694776200586141...",en,268676434,737889402,@RanaAyyub Anupam Babu deserves Padma Award if...,False,[695261334234124288],everyone,2016-02-04T15:03:14.000Z,NaN,"{'mentions': [{'start': 0, 'end': 10, 'usernam...",694776200586141696,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",695261334234124288,NaN,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,NaN,"[0.30925182, 0.5520057, 0.13874246]",1


In [284]:
# def convs_x(df):
subsetdf[subsetdf.conversation_id=='1303613483024396288']

,referenced_tweets,lang,in_reply_to_user_id,author_id,text,possibly_sensitive,edit_history_tweet_ids,reply_settings,created_at,context_annotations,entities,conversation_id,public_metrics,id,attachments,edit_controls,geo,withheld,score,label
653,NaN,en,NaN,1252454205324800000,Ex BJP minister CH lal singh and his brother t...,False,[1303613483024396288],everyone,2020-09-09T08:37:28.000Z,"[{'domain': {'id': '10', 'name': 'Person', 'de...","{'annotations': [{'start': 3, 'end': 5, 'proba...",1303613483024396288,"{'retweet_count': 42, 'reply_count': 11, 'like...",1303613483024396288,"{'media_keys': ['3_1303613430360760320', '3_13...","{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,NaN,"[0.79797286, 0.19311304, 0.008914174]",0
534587,NaN,en,NaN,1252454205324800000,Ex BJP minister CH lal singh and his brother t...,False,[1303613483024396288],everyone,2020-09-09T08:37:28.000Z,"[{'domain': {'id': '10', 'name': 'Person', 'de...","{'urls': [{'start': 277, 'end': 300, 'url': 'h...",1303613483024396288,"{'retweet_count': 42, 'reply_count': 11, 'like...",1303613483024396288,"{'media_keys': ['3_1303613430360760320', '3_13...","{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,NaN,"[0.79797286, 0.19311304, 0.008914174]",0
551038,NaN,en,NaN,1252454205324800000,Ex BJP minister CH lal singh and his brother t...,False,[1303613483024396288],everyone,2020-09-09T08:37:28.000Z,"[{'domain': {'id': '10', 'name': 'Person', 'de...","{'mentions': [{'start': 85, 'end': 97, 'userna...",1303613483024396288,"{'retweet_count': 42, 'reply_count': 11, 'like...",1303613483024396288,"{'media_keys': ['3_1303613430360760320', '3_13...","{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,NaN,"[0.79797286, 0.19311304, 0.008914174]",0
887780,NaN,en,NaN,1252454205324800000,Ex BJP minister CH lal singh and his brother t...,False,[1303613483024396288],everyone,2020-09-09T08:37:28.000Z,"[{'domain': {'id': '10', 'name': 'Person', 'de...","{'mentions': [{'start': 85, 'end': 97, 'userna...",1303613483024396288,"{'retweet_count': 42, 'reply_count': 11, 'like...",1303613483024396288,"{'media_keys': ['3_1303613430360760320', '3_13...","{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,NaN,"[0.79797286, 0.19311304, 0.008914174]",0


In [253]:
involved_users[0][111]

{'username': 'ASUR____',
 'profile_image_url': 'https://pbs.twimg.com/profile_images/1166340227297726464/SkAVLHCS_normal.jpg',
 'pinned_tweet_id': '834715805610749952',
 'created_at': '2009-08-27T17:59:55.000Z',
 'name': 'ˢꚝ࿆',
 'description': '⠀\n⠀\n⠀\n⠀\n⠀\n⠀\n⠀\n⠀\n⠀\n⠀⠀⠀ \n⠀\n⠀\n⠀\n⠀\n⠀\n⠀\n⠀\n⠀\n⠀\n⠀\n⠀\n⠀',
 'id': '69358010',
 'verified': False,
 'protected': False,
 'public_metrics': {'followers_count': 266,
  'following_count': 111,
  'tweet_count': 9558,
  'listed_count': 4}}

In [ ]:
captured_tweets = []
en = 0
for tweet in alice[0]: 
    if tweet['lang'] == 'en' or tweet['lang'] == 'und' or tweet['lang'] == 'et': # we only have the capabilities for english at the moment 
            record_text = tweet['text']
            record_text = preprocess(record_text)
            encoded_input = tokenizer(record_text, return_tensors='pt')
            try:
                output = model(**encoded_input)
            else:
                print(tweet['text'])
            scores = output[0][0].detach().numpy()
            scores = softmax(scores)
            record = {}
            record['text'] = tweet['text']
            record['score']  = scores 
            record['conversation_id'] = tweet['conversation_id']
            record['author_id'] = tweet['author_id']
            record['created_at'] = tweet['created_at']
            record['label'] = np.argmax(scores)
            captured_tweets.append(record)
    else:
        continue
            
#     langs.append(tweet['lang'])

In [194]:
capTweets = pd.DataFrame(captured_tweets)
capTweets.to_csv('./data/caps_tweets.csv')

In [198]:
capTweets.iloc[2]['text']

'@RanaAyyub Anupam Babu deserves Padma Award if he has been able to disturb you. Left terror in education institutions will end. get ready...'

In [237]:
rana = capTweets[capTweets.author_id=='268676434']
for i in rana:
    if ra:
        print(i)
        break;

In [232]:
capTweets[capTweets.author_id=='268676434'].iloc[0]['text']

'@sky_phd 🙏'

In [ ]:
id_overall_imp = {}
for usr in tqdm(list(capTweets.author_id.unique())):
    label = capTweets[capTweets.author_id==usr].score.mean().argmax()
    id_overall_imp[usr] = label

In [205]:
data = HeteroData() # directionanlity usage 
data['conversations'].x = torch.Tensor(np.stack(convs['score'].values, axis=0))
data['conversations'].y = torch.Tensor(convs['label'].values)
data['replies'].x  = torch.Tensor(np.stack(replies['score'].values, axis=0))
data['replies'].y = torch.Tensor(replies['label'].values)

16657

In [122]:
users = pd.DataFrame(involved_users[0])

In [144]:
users[users.id == '1398533566619811841'].iloc[0]

username                                               HarshReality891
protected                                                        False
id                                                 1398533566619811841
public_metrics       {'followers_count': 15, 'following_count': 121...
verified                                                         False
profile_image_url    https://pbs.twimg.com/profile_images/159633342...
description                                                           
created_at                                    2021-05-29T06:56:14.000Z
name                                                            Olivia
pinned_tweet_id                                    1607777996848627717
url                                                                NaN
location                                                           NaN
entities                                                           NaN
withheld                                                           NaN
Name: 

In [183]:
uniUserDf = pd.DataFrame(columns=['username', 'protected', 'id', 'public_metrics', 'verified',
       'profile_image_url', 'description', 'created_at', 'name',
       'pinned_tweet_id', 'url', 'location', 'entities', 'withheld'])
for idx in users.id.unique():
    record = users[users.id==idx].iloc[0]
    uniUserDf = uniUserDf.append(record)
    del record 

In [358]:
convs = subsetdf[subsetdf.author_id=='268676434']
convs = convs.reset_index()
convs = convs.loc[convs.id.astype(str).drop_duplicates().index]
convs_ids = list(convs.id)
convsidx_ids = list(convs.conversation_id)
replies = subsetdf[~subsetdf.id.isin(convs_ids)]
replies = replies[replies.conversation_id.isin(convsidx_ids)]
replies = replies.loc[replies.id.astype(str).drop_duplicates().index]
replies = replies.reset_index()


In [381]:
pd.isna(replies[replies.referenced_tweets.isnull()])

,index,referenced_tweets,lang,in_reply_to_user_id,author_id,text,possibly_sensitive,edit_history_tweet_ids,reply_settings,created_at,...,entities,conversation_id,public_metrics,id,attachments,edit_controls,geo,withheld,score,label
26,29,NaN,en,268676434,9375282,@RanaAyyub Dear Rana - I'm messaging from @ris...,False,[1451521804271554560],mentionedUsers,2021-10-22T12:12:20.000Z,...,"{'mentions': [{'start': 0, 'end': 10, 'usernam...",1451521804271554560,"{'retweet_count': 35, 'reply_count': 2, 'like_...",1451521804271554560,NaN,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,NaN,"[0.011182841, 0.7917029, 0.19711427]",1
435,462,NaN,en,NaN,1054216531553398785,"Raja Vemula, my younger son, is now an Advocat...",False,[1339987026070831104],everyone,2020-12-18T17:32:57.000Z,...,"{'annotations': [{'start': 0, 'end': 10, 'prob...",1339987026070831104,"{'retweet_count': 4651, 'reply_count': 672, 'l...",1339987026070831104,NaN,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,NaN,"[0.0037004238, 0.071906626, 0.92439294]",2
1109,1181,NaN,en,NaN,805476235690254338,"Happy Birthday @RanaAyyub, May Allah bless u w...",False,[1388235994231570432],everyone,2021-04-30T20:57:07.000Z,...,"{'urls': [{'start': 177, 'end': 200, 'url': 'h...",1388235994231570432,"{'retweet_count': 2, 'reply_count': 5, 'like_c...",1388235994231570432,{'media_keys': ['3_1388235979668873216']},"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,NaN,"[0.0036647082, 0.012483795, 0.9838515]",2


In [438]:
replies[replies]

,index,referenced_tweets,lang,in_reply_to_user_id,author_id,text,possibly_sensitive,edit_history_tweet_ids,reply_settings,created_at,...,entities,conversation_id,public_metrics,id,attachments,edit_controls,geo,withheld,score,label
0,1,"[{'type': 'replied_to', 'id': '159701023284815...",und,117673128,268676434,@sky_phd 🙏,False,[1597083220184428544],everyone,2022-11-28T04:21:26.000Z,...,"{'mentions': [{'start': 0, 'end': 8, 'username...",1597010232848154624,"{'retweet_count': 0, 'reply_count': 2, 'like_c...",1597083220184428544,NaN,"{'edits_remaining': 5, 'is_edit_eligible': Fal...",NaN,NaN,"[0.04223864, 0.43336734, 0.524394]",2
1,5,"[{'type': 'quoted', 'id': '694773794045558784'}]",en,NaN,268676434,Wonder what anupam babu has to offer on this a...,False,[694776200586141696],everyone,2016-02-03T06:55:29.000Z,...,"{'urls': [{'start': 70, 'end': 93, 'url': 'htt...",694776200586141696,"{'retweet_count': 57, 'reply_count': 30, 'like...",694776200586141696,NaN,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,NaN,"[0.40171865, 0.56999224, 0.028289113]",1
2,27,"[{'type': 'replied_to', 'id': '145152180427155...",en,9375282,268676434,@LauraOliver @risj_oxford @TIME Sure,False,[1451530465354010629],mentionedUsers,2021-10-22T12:46:45.000Z,...,"{'mentions': [{'start': 0, 'end': 12, 'usernam...",1451521804271554560,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",1451530465354010629,NaN,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,NaN,"[0.02509196, 0.817922, 0.15698609]",1
3,31,NaN,en,NaN,268676434,Also watch this debate to see footage of what ...,False,[835102933251461120],everyone,2017-02-24T12:23:49.000Z,...,"{'hashtags': [{'start': 73, 'end': 80, 'tag': ...",835102933251461120,"{'retweet_count': 23, 'reply_count': 11, 'like...",835102933251461120,NaN,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,NaN,"[0.09088399, 0.8842918, 0.024824103]",1
4,39,"[{'type': 'quoted', 'id': '781533698185244672'}]",und,NaN,268676434,errr !! https://t.co/g1sniqPwTz,False,[781539402891857920],everyone,2016-09-29T17:01:28.000Z,...,"{'urls': [{'start': 8, 'end': 31, 'url': 'http...",781539402891857920,"{'retweet_count': 16, 'reply_count': 43, 'like...",781539402891857920,NaN,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,NaN,"[0.017667584, 0.6158092, 0.36652324]",1
5,74,NaN,en,NaN,268676434,ARGUMENT\nDo France’s Intellectuals Have a Mus...,False,[553645343095521280],everyone,2015-01-09T20:11:50.000Z,...,"{'annotations': [{'start': 12, 'end': 17, 'pro...",553645343095521280,"{'retweet_count': 7, 'reply_count': 22, 'like_...",553645343095521280,NaN,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,NaN,"[0.34497067, 0.64241964, 0.0126096355]",1
6,76,NaN,en,NaN,268676434,"Coming from the editor of ""Centre Right india""...",False,[463228908263833600],everyone,2014-05-05T08:09:12.000Z,...,"{'mentions': [{'start': 80, 'end': 94, 'userna...",463228908263833600,"{'retweet_count': 8, 'reply_count': 19, 'like_...",463228908263833600,NaN,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,NaN,"[0.051452763, 0.6168594, 0.33168787]",1
7,78,"[{'type': 'quoted', 'id': '718427886970105856'}]",en,NaN,268676434,So the J&amp;K police who until now were heroe...,False,[718489259749978112],everyone,2016-04-08T17:22:43.000Z,...,"{'hashtags': [{'start': 106, 'end': 118, 'tag'...",718489259749978112,"{'retweet_count': 94, 'reply_count': 122, 'lik...",718489259749978112,NaN,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,NaN,"[0.7011347, 0.29038075, 0.008484542]",0
9,180,NaN,en,NaN,268676434,Smriti Iranis gives statement on the humiliati...,False,[705252587155533824],everyone,2016-03-03T04:44:54.000Z,...,"{'annotations': [{'start': 0, 'end': 12, 'prob...",705252587155533824,"{'retweet_count': 106, 'reply_count': 22, 'lik...",705252587155533824,NaN,"{'edits_remaining': 5, 'is_edit_eligible': Tru...",NaN,NaN,"[0.66125584, 0.32570732, 0.013036878]",0
10,195,NaN,en,NaN,268676434,Delighted to be in this stellar list of speake...,False

In [475]:
a = replies[replies.id=='1451548979687092224']
a['referenced_tweets'].values[0][0][]

{'type': 'replied_to', 'id': '1451521804271554560'}

In [ ]:
replies[replies.id=='replies[replies.id=='1451548979687092224']']

In [426]:
idx = replies.iloc[96]['referenced_tweets'][0]['id']
convs[convs.id==idx].index.values[0]

7

In [467]:
def mapped_idex(df):
    a = np.array(df['referenced_tweets'])
    if pd.isna(a).all():
        idee = df['author_id']
        mapped = convs[convs.in_reply_to_user_id==idee].index.values[0]
        return mapped
    if len(df['referenced_tweets'])>1:
        idex = df['referenced_tweets'][1]['id']
        mapped = convs[convs.id==idex].index.values[0]
        return mapped
    idex = df['referenced_tweets'][0]['id']
    if df['referenced_tweets'][0]['type'] == 'quoted':
        print(df['id'])
        mapped = 0
        return mapped
    print(df['id'])
    mapped = convs[convs.id==idex].index.values
    if len(mapped) > 0:
        mapped = mapped[0]
    else:
        mapped = replies[convs.id==idex].index.values
    return mapped 
mapped_idex(replies.iloc[15])

1

In [468]:
replies_to_convo = np.zeros((2,len(replies)))
for i in range(len(replies)):
        replies_to_convo[0][i] = i
        replies_to_convo[1][i] = mapped_idex(replies.iloc[i])
replies_to_convo

1597290445016510464
1597092138692800514
1597010232848154624
695261334234124288
694930401446367232
694913944511614976
694829457522077696
694805696026574848
694799502364905473
694789577068810240
694787148298489857
694781486013108224
694780248441823236
694778514730586113
694778289408180224
694777796204314624
694777786968330240
694777687471075328
694777468742270976
694777100734222336
694776998242185216
694776979376119808
694776804050075648
1451555503344132098
1451548979687092224


IndexError: index 0 is out of bounds for axis 0 with size 0

In [293]:
conv_x = torch.Tensor(convs['score'].values)
conv_x

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [186]:
uniUserDf.to_csv('./data/unique_users_rana.csv')

,username,protected,id,public_metrics,verified,profile_image_url,description,created_at,name,pinned_tweet_id,url,location,entities,withheld
0,HarshReality891,False,1398533566619811841,"{'followers_count': 15, 'following_count': 121...",False,https://pbs.twimg.com/profile_images/159633342...,,2021-05-29T06:56:14.000Z,Olivia,1607777996848627717,NaN,NaN,NaN,NaN
1,RanaAyyub,False,268676434,"{'followers_count': 1556144, 'following_count'...",True,https://pbs.twimg.com/profile_images/160214177...,Journalist. Social Justice. Opinions at @washi...,2011-03-19T07:21:54.000Z,Rana Ayyub,1495411439938732034,https://t.co/yXld4rWeun,"Mumbai, India","{'url': {'urls': [{'start': 0, 'end': 23, 'url...",NaN
2,sky_phd,False,117673128,"{'followers_count': 1784, 'following_count': 5...",False,https://pbs.twimg.com/profile_images/160419319...,"Senior Scientist, Inventor, Nanotechnology, Po...",2010-02-26T07:19:26.000Z,"Santosh Yadav, Ph.D.",1193083415236661248,NaN,"Cleveland, OH",NaN,NaN
3,ashoktewari,False,48441905,"{'followers_count': 774, 'following_count': 14...",False,https://pbs.twimg.com/profile_images/797295818...,"Right Winger, There are only 2 sides, Right or...",2009-06-18T18:31:50.000Z,Ashok Tewari,720223982922764290,NaN,NaN,"{'description': {'hashtags': [{'start': 133, '...",NaN
4,PratMangal,False,737889402,"{'followers_count': 351, 'following_count': 21...",False,https://pbs.twimg.com/profile_images/158478967...,,2012-08-05T03:11:59.000Z,Prateek Mangal,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,BhaweshTiwari,False,117288618,"{'followers_count': 363, 'following_count': 85...",False,https://pbs.twimg.com/profile_images/716917333...,A Non Resident Bihari!A hardcore SANGHI.Great ...,2010-02-25T03:47:09.000Z,Bhawesh T@Wellington,NaN,NaN,"Wellington, New Zealand",NaN,NaN
105,drvickykandal,False,701948731600470016,"{'followers_count': 5, 'following_count': 78, ...",False,https://pbs.twimg.com/profile_images/720826692...,,2016-02-23T01:56:34.000Z,vicky,NaN,NaN,NaN,NaN,NaN
106,kajaruddinFarji,False,303633927,"{'followers_count': 5, 'following_count': 126,...",False,https://abs.twimg.com/sticky/default_profile_i...,,2011-05-23T06:38:58.000Z,satya,NaN,NaN,NaN,NaN,NaN
107,vkbaxi,False,105892645,"{'followers_count': 77, 'following_count': 555...",False,https://pbs.twimg.com/profile_images/133580760...,"University Vice Chair, Fmr Treasurer at major ...",2010-01-17T21:09:15.000Z,V K Baxi,NaN,NaN,NaN,NaN,NaN


In [178]:
uniUserDf.append(users.iloc[0])


,username,protected,id,public_metrics,verified,profile_image_url,description,created_at,name,pinned_tweet_id,url,location,entities,withheld
0,HarshReality891,False,1398533566619811841,"{'followers_count': 15, 'following_count': 121...",False,https://pbs.twimg.com/profile_images/159633342...,,2021-05-29T06:56:14.000Z,Olivia,1607777996848627717,NaN,NaN,NaN,NaN


In [179]:
uniUserDf

,username,protected,id,public_metrics,verified,profile_image_url,description,created_at,name,pinned_tweet_id,url,location,entities,withheld
